In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'15',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'key',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd

In [ ]:
# This allows you to see all the clumns, not just like 15
pd.set_option('display.max_columns', None)

#This normalizes the data and assigns it to a dataframe
df = pd.json_normalize(data['data'])

In [ ]:
df['timestamp'] = pd.to_datetime('now')

In [ ]:
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'15',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': 'key',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
      print(e)
    
    # This allows you to see all the clumns, not just like 15
    pd.set_option('display.max_columns', None)

    #This normalizes the data and assigns it to a dataframe
    df = pd.json_normalize(data['data'])

    df['timestamp'] = pd.to_datetime('now')
    df
    
    if not os.path.isfile(r'File Path'):
        df.to_csv(r'File Path', header = 'column_names')
    else:
        df.to_csv(r'File Path', mode = 'a', header = False)
    

In [ ]:
import os
from time import time
from time import sleep

# max pulls
for i in range(333):
    api_runner()
    print('API Runner completed')
    sleep(60) #sleep for 1 minute
exit()

In [ ]:
df_csv = pd.read_csv(r'File Path')
df_csv

In [ ]:
# Reformat numerical values

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df_csv

In [ ]:
df3 = df_csv.groupby('name', sort=False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d' ]].mean()
df3

In [ ]:
df4 = df3.stack()
df4

In [ ]:
df5 = df4.to_frame(name='values')
df5

In [ ]:
index = pd.Index(range(df5.count()['values']))
df6 = df5.reset_index()
df6

In [ ]:
df7 = df6.rename(columns={'level_1': 'percent_change'})
df7

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df7['percent_change'] = df7['percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'],['1h','24h','7d','30d','60d','90d'])

In [ ]:
df7
sns.catplot(x = 'percent_change', y = 'values', hue = 'name', data = df7, kind = 'point')

In [ ]:
df8 = df_csv[['name','quote.USD.price','timestamp']]
df8 = df8.query("name == 'Bitcoin'")
df8

In [ ]:
sns.set_theme(style = 'darkgrid')
sns.lineplot(x='timestamp', y = 'quote.USD.price', data = df8)